In [6]:
#!/usr/bin/env python3
# subscriber.py
import cv2, zmq, numpy as np, time, threading, queue, traceback, sys, os,csv
from collections import deque, defaultdict

# ---------- Config ----------
ZMQ_ADDR = "tcp://localhost:5555"
SUB_TOPICS = [b"kreo1", b"kreo2"]
FPS_WINDOW = 1.0        # seconds for fps moving window
DISPLAY_FPS = 20
VISUALIZE = True     # show tiled view window

STATIC_TAG_IDS = [0,1,2,3]
TAG_POSITIONS = {
    0: np.array([0.9, 0.0, 0.0], dtype=float),
    1: np.array([0.0, 0.0, 0.0], dtype=float),
    2: np.array([0.9, 0.9, 0.0], dtype=float),
    3: np.array([0.0, 0.9, 0.0], dtype=float)
}
TAG_SIZES = {0: 0.099, 1: 0.096, 2: 0.096, 3: 0.096, 4: 0.096, 5: 0.096}
CALIB_FRAMES = 30
CALIB_DIR = "../calibration/"

# ---------- CSV LOGGING SETUP ----------
# Create a file with a timestamp to avoid overwriting
log_filename = f"data_log_{int(time.time())}.csv"
csv_file = open(log_filename, mode='w', newline='')
csv_writer = csv.writer(csv_file)

# Write the header
# ts: system timestamp
# ball: x, y, z world coordinates
# tag4: x, y, z world coordinates
# tag5: x, y, z world coordinates
header = [
    "timestamp", 
    "ball_x", "ball_y", "ball_z", 
    "tag4_x", "tag4_y", "tag4_z", 
    "tag5_x", "tag5_y", "tag5_z"
]
csv_writer.writerow(header)
print(f"[Logging] Started logging to {log_filename}")

# ---------------- APRILTAG CONFIG ----------------
DICT_TYPE = cv2.aruco.DICT_APRILTAG_36h11
def create_april_detector():
    """Setup AprilTag detector with tuned parameters."""
    aruco_dict = cv2.aruco.getPredefinedDictionary(DICT_TYPE)
    params = cv2.aruco.DetectorParameters()
    params.adaptiveThreshWinSizeMin = 3
    params.adaptiveThreshWinSizeMax = 35
    params.adaptiveThreshWinSizeStep = 2
    params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX
    params.cornerRefinementWinSize = 7
    params.cornerRefinementMaxIterations = 50
    params.cornerRefinementMinAccuracy = 0.01
    params.minMarkerPerimeterRate = 0.02
    params.maxMarkerPerimeterRate = 6.0
    params.polygonalApproxAccuracyRate = 0.02
    params.adaptiveThreshConstant = 7
    return cv2.aruco.ArucoDetector(aruco_dict, params)

# color thresholds (you gave these)
orange_hsvVals = {'hmin': 0,  'smin': 94,  'vmin': 156, 'hmax': 12,  'smax': 255, 'vmax': 255}
purple_hsvVals = {'hmin': 113,'smin': 78,  'vmin': 3,   'hmax': 129, 'smax': 255, 'vmax': 255}

# detector parameters
MIN_AREA = 150    # min contour area to accept (tune if needed)
MAX_AREA = 20000  # max area (avoid very large blobs)
CIRCULARITY_MIN = 0.25  # min circularity to accept (lower because paper balls can deform)
ASPECT_RATIO_MAX = 2.0   # reject extremely elongated blobs
MAX_DETECTIONS_PER_CAM = 12  # safety limit

# ---------- Helpers ----------
def fmt_ts(ts):
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ts)) + f".{int((ts%1)*1000):03d}"

def recv_latest(sub):
    msg = None
    while True:
        try:
            msg = sub.recv_multipart(flags=zmq.NOBLOCK)
        except zmq.Again:
            break
    return msg

def update_fps(camera, cam_ts):
    dq = fps_windows[camera]
    dq.append(cam_ts)
    # pop older than window
    while dq and (cam_ts - dq[0]) > FPS_WINDOW:
        dq.popleft()
    fps = len(dq) / FPS_WINDOW
    return fps

def load_camera_calib(cam_name):
    path = os.path.join(CALIB_DIR, f'camera_calibration_{cam_name}.npz')
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    calib = np.load(path)
    camera_matrix = calib["cameraMatrix"]
    dist_coeffs = calib['distCoeffs']
    print("[INFO] Loaded calibrated camera parameters")
    return camera_matrix, dist_coeffs

def build_tag_world_map_from_centers(tag_centers, tag_sizes):
    """
    Returns dict: id -> 4x3 corners
    ([-half, +half],[+half,+half],[+half,-half],[-half,-half]) in tag local XY plane, then added to center.
    Assumes tag plane normal is +Z and tag axes align with world XY.
    """
    out = {}
    for tid, center in tag_centers.items():
        size = tag_sizes.get(tid, tag_sizes.get(1))  # fallback to 1's size
        half = float(size) / 2.0
        # ordering matches your partial_triangulation objp:
        local = np.array([
            [-half,  half, 0.0],
            [ half,  half, 0.0],
            [ half, -half, 0.0],
            [-half, -half, 0.0],
        ], dtype=np.float64)
        corners_world = (local + center.reshape(1,3)).astype(np.float64)
        out[tid] = corners_world  # 4x3
    return out

TAG_WORLD_MAP = build_tag_world_map_from_centers(TAG_POSITIONS, TAG_SIZES)

# ---------- Color masking helpers ----------
def hsv_mask_from_vals(bgr_img, hsvVals):
    """Return binary mask from HSV thresholds dict."""
    hsv = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)
    lower = np.array([hsvVals['hmin'], hsvVals['smin'], hsvVals['vmin']], dtype=np.uint8)
    upper = np.array([hsvVals['hmax'], hsvVals['smax'], hsvVals['vmax']], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower, upper)
    # small blur to reduce speckle
    mask = cv2.medianBlur(mask, 5)
    return mask

def postprocess_mask(mask):
    """Morphological clean-up."""
    # open then close
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    m = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kernel, iterations=1)
    return m

def find_candidate_contours(mask):
    """Return list of contours filtered by area and shape heuristics."""
    if mask is None:
        return []
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidates = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < MIN_AREA or area > MAX_AREA:
            continue
        perim = cv2.arcLength(c, True)
        if perim <= 0:
            continue
        circularity = 4 * np.pi * area / (perim * perim)
        x,y,w,h = cv2.boundingRect(c)
        aspect = float(w)/float(h) if h>0 else 0.0
        # accept if roughly circular-ish or moderate area even if circularity low
        if circularity >= CIRCULARITY_MIN or (0.5*min(w,h) > 5 and area > (MIN_AREA*2)):
            if aspect <= ASPECT_RATIO_MAX:
                candidates.append((c, area, (int(x),int(y),int(w),int(h))))
    # sort by area desc
    candidates.sort(key=lambda d: d[1], reverse=True)
    return candidates

def estimate_pose_apriltag(corners, tag_size, cam_mtx, cam_dist):
    half = tag_size / 2.0
    objp = np.array([
        [-half,  half, 0.0],
        [ half,  half, 0.0],
        [ half, -half, 0.0],
        [-half, -half, 0.0]
    ], dtype=np.float32)

    imgp = corners.reshape(4,2).astype(np.float32)

    ok, rvec, tvec = cv2.solvePnP(
        objp, imgp, cam_mtx, cam_dist,
        flags=cv2.SOLVEPNP_ITERATIVE
    )

    if not ok:
        raise RuntimeError("solvePnP failed")

    R, _ = cv2.Rodrigues(rvec)
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = tvec.reshape(3)
    return T

# APRILTAG DETECTOR THREAD
class AprilTagThread(threading.Thread):
    def __init__(self, cam_name, frame_queue, detect_cache, lock, calibrator, shared_robot_poses):
        super().__init__(daemon=True)
        self.cam_name = cam_name
        self.frame_queue = frame_queue
        self.detect_cache = detect_cache
        self.lock = lock
        self.detector = create_april_detector()
        self.stop_flag = False
        self.calibrator = calibrator
        self.shared_robot_poses = shared_robot_poses
    
    def run(self):
        print(f"[{self.cam_name}] AprilTagThread started")
        while not self.stop_flag:
            try:
                frame,ts = self.frame_queue.get(timeout=0.1)
            except queue.Empty:
                continue
            try:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                corners, ids, _ = self.detector.detectMarkers(gray)
                det={"corners":corners,"ids":ids,"ts":ts,"det_time":time.time()}


                with self.lock:
                    if self.cam_name not in self.detect_cache:
                        self.detect_cache[self.cam_name]={}
                    self.detect_cache[self.cam_name]["tags"]=det

                if ids is not None:
                    self.calibrator.add_detection(self.cam_name,ids,corners,ts)
                    self.calibrator.try_compute_extrinsic(self.cam_name)

                    if self.cam_name in self.calibrator.extrinsics:
                        try:
                            K, dist = self.calibrator.load_intrinsics(self.cam_name)
                            for i,idarr in enumerate(ids):
                                tid = int(idarr[0])
                                if tid in (4,5):
                                    corners_i = np.array(corners[i]).reshape(4,2)
                                    # T = estimate_pose_apriltag(corners_i, TAG_SIZES[tid], K, dist)
                                    # # T maps tag -> camera: X_cam = R*X_tag + t  (matching estimate_pose_apriltag)
                                    # R = T[:3,:3]
                                    # t = T[:3,3].reshape(3,)
                                    # X_cam = t  # camera coordinates of tag origin
                                    # # convert to world using calibrator (X_world = R_cam^T @ (X_cam - t_cam))
                                    # X_world = self.calibrator.cam_to_world(self.cam_name, X_cam)
                                    # with self.lock:
                                    #     # store latest pose for this tag id (world coords)
                                    #     print(f"tid: {tid}, pos: {X_world}")
                                    #     self.shared_robot_poses[tid] = {'world_pos': X_world, 'cam': self.cam_name, 'ts': ts, 'det_time': time.time(), 'reproj_src':'estimate_pose_apriltag'}

                                    T_tag_cam = estimate_pose_apriltag(corners_i, TAG_SIZES[tid], K, dist)
                                    R_tag_cam = T_tag_cam[:3,:3]
                                    t_tag_cam = T_tag_cam[:3,3].reshape(3,)   # tag origin in camera frame

                                    # compute the 3D positions of the 4 tag corners in camera frame
                                    half = TAG_SIZES[tid] / 2.0
                                    objp = np.array([
                                        [-half,  half, 0.0],
                                        [ half,  half, 0.0],
                                        [ half, -half, 0.0],
                                        [-half, -half, 0.0]
                                    ], dtype=np.float64)   # 4x3
                                    cam_corners = (R_tag_cam @ objp.T).T + t_tag_cam.reshape(1,3)  # 4x3

                                    # convert tag origin and corners to world coordinates
                                    tag_origin_world = self.calibrator.cam_to_world(self.cam_name, t_tag_cam)
                                    world_corners = self.calibrator.cam_to_world(self.cam_name, cam_corners)  # 4x3

                                    # diagnostic: mean Z of corners
                                    mean_corner_z = float(np.mean(world_corners[:,2]))

                                    # If mean_corner_z is negative (i.e. tag plane ends up below ground with negative sign),
                                    # this suggests an orientation/sign flip — correct the reported Z (safe heuristic).
                                    corrected_tag_origin_world = tag_origin_world.copy()
                                    if mean_corner_z < 0:
                                        # flip Z sign: this handles cases where tag normal is inverted
                                        corrected_tag_origin_world[2] = abs(corrected_tag_origin_world[2])
                                        print(f"[WARN] {self.cam_name} tag{tid} mean_corner_z negative ({mean_corner_z:.4f}). Flipping Z to {corrected_tag_origin_world[2]:.4f}")

                                    # Final world position: but we also know robot tag nominal height if needed
                                    # (you said robot tag center is at +0.075 m), so we can prefer that value if close
                                    nominal_tag_height = 0.075
                                    # if the measured z differs by > 0.05m, we prefer the nominal (optional)
                                    if abs(corrected_tag_origin_world[2] - nominal_tag_height) > 0.05:
                                        # warn but do not overwrite automatically; instead print for debugging
                                        print(f"[INFO] {self.cam_name} tag{tid} measured z {corrected_tag_origin_world[2]:.3f}, nominal {nominal_tag_height:.3f}")

                                    # choose the corrected origin as final output
                                    with self.lock:
                                        self.shared_robot_poses[tid] = {
                                            'world_pos': corrected_tag_origin_world,
                                            'cam': self.cam_name,
                                            'ts': ts,
                                            'det_time': time.time(),
                                            'reproj_src': 'estimate_pose_apriltag',
                                            'mean_corner_z': mean_corner_z,
                                            'cam_t': t_tag_cam.copy()
                                        }
                                        print(f"tid: {tid}, pos: {corrected_tag_origin_world}")
                        except Exception as e:
                            print(f"[{self.cam_name}] robot pose compute error:", e)
                            traceback.print_exc()

            except Exception as e:
                print(f"[ERROR-{self.cam_name}] AprilTag Detection exception:", e)
                traceback.print_exc()

        print(f"[DETECT-{self.cam_name}] AprilTag Detector thread stopped")

    def stop(self):
        self.stop_flag = True


# BALL DETECTOR THREAD
class BallThread(threading.Thread):
    def __init__(self, cam_name, frame_queue, detect_cache, lock, shared_camera_candidates):
        super().__init__(daemon=True)
        self.cam_name = cam_name
        self.frame_queue = frame_queue
        self.detect_cache = detect_cache
        self.lock = lock
        self.stop_flag = False
        self.shared_camera_candidates = shared_camera_candidates
    
    def run(self):
        print(f"[{self.cam_name}] BallThread started")
        while not self.stop_flag:
            try:
                frame,ts=self.frame_queue.get(timeout=0.1)
            except queue.Empty:
                continue
            try:
                mo=hsv_mask_from_vals(frame,orange_hsvVals)
                mp=hsv_mask_from_vals(frame,purple_hsvVals)

                mc = cv2.bitwise_or(mo,mp)
                mc = postprocess_mask(mc)

                cand = find_candidate_contours(mc)

                dets = []
                for c,area,(x,y,w,h) in cand[:MAX_DETECTIONS_PER_CAM]:
                    M = cv2.moments(c)
                    if M["m00"] != 0:
                        cx = int(M["m10"]/M["m00"]); cy = int(M["m01"]/M["m00"])
                    else:
                        cx = x + w//2; cy = y + h//2
                    
                    s_or = int(np.count_nonzero(mo[y:y+h,x:x+w])) if mo is not None else 0
                    s_pu = int(np.count_nonzero(mp[y:y+h,x:x+w])) if mp is not None else 0

                    if s_or>s_pu and s_or>0:
                        col="orange"
                    elif s_pu>s_or and s_pu>0:
                        col="purple"
                    else:
                        hsv_roi=cv2.cvtColor(frame[y:y+h,x:x+w],cv2.COLOR_BGR2HSV)
                        mean_h=int(np.mean(hsv_roi[:,:,0]))
                        if orange_hsvVals["hmin"]<=mean_h<=orange_hsvVals["hmax"]:
                            col="orange"
                        elif purple_hsvVals["hmin"]<=mean_h<=purple_hsvVals["hmax"]:
                            col="purple"
                        else:
                            col="unknown"

                    dets.append({
                        "bbox":(x,y,w,h),
                        "centroid":(cx,cy),
                        "area":float(area),
                        "color":col,
                        "ts":ts,
                        "det_time":time.time()
                    })
                with self.lock:
                    if self.cam_name not in self.detect_cache:
                        self.detect_cache[self.cam_name]={}
                    self.detect_cache[self.cam_name]["balls"] = dets
                    self.shared_camera_candidates[self.cam_name] = [{"centroid":d["centroid"], "area":d["area"], "color":d["color"], "ts":d["ts"]} for d in dets]
            except Exception as e:
                print(f"[ERROR-{self.cam_name}] ball detection exception:", e)
                traceback.print_exc()

        print(f"[{self.cam_name}] BallDetectorThread stopped")

    def stop(self):
        self.stop_flag = True


# Calibrator Class
class StaticCalibrator:
    def __init__(self, tag_world_map, tag_size_map):
        self.tag_world_map = tag_world_map
        self.tag_size_map = tag_size_map
        self.obs = defaultdict(list)   # cam_name -> list of (id, corners(4x2), ts)
        self.extrinsics = {}           # cam_name -> {'rvec','tvec','R'}
        self.frame_count = defaultdict(int)
        self.P_cache = {}              # cam_name -> projection matrix K@[R|t]
        self.K_cache = {}              # cam_name -> camera matrix
        self.dist_cache = {}           # cam_name -> dist coeffs

    def load_intrinsics(self, cam_name):
        if cam_name in self.K_cache:
            return self.K_cache[cam_name], self.dist_cache[cam_name]
        # rely on your existing loader function location & naming
        camera_matrix, dist_coeffs = load_camera_calib(cam_name)
        self.K_cache[cam_name] = camera_matrix
        self.dist_cache[cam_name] = dist_coeffs
        return camera_matrix, dist_coeffs

    def add_detection(self, cam_name, ids, corners, ts):
        """
        ids: Nx1 array from detector (or None)
        corners: list/array Nx1x4x2 (same as cv2.aruco.detectMarkers output) or list of Nx4x2
        ts: timestamp
        """
        if ids is None:
            return
        self.frame_count[cam_name] += 1
        # normalize corners into shape (N,4,2)
        for i, idarr in enumerate(ids):
            tid = int(idarr[0])
            if tid in STATIC_TAG_IDS:
                c = np.array(corners[i]).reshape(4,2).astype(np.float64)
                self.obs[cam_name].append((tid, c, ts))

    def try_compute_extrinsic(self, cam_name):
        if cam_name in self.extrinsics:
            return True

        if self.frame_count.get(cam_name, 0) < CALIB_FRAMES:
            return False

        if cam_name == "kreo1":
            target_tag = 2
        elif cam_name == "kreo2":
            target_tag = 1
        else:
            return False

        obs_list = list(reversed(self.obs.get(cam_name, [])))

        use_corners = None
        for (tid, corners, ts) in obs_list:
            if int(tid) == int(target_tag):
                use_corners = corners.reshape(4,2).astype(np.float64)
                break

        if use_corners is None:
            return False

        try:
            K, dist = self.load_intrinsics(cam_name)
        except Exception as e:
            return False

        obj_corners = np.array(self.tag_world_map[target_tag], dtype=np.float64)

        ok, rvec, tvec = cv2.solvePnP(
            obj_corners,
            use_corners,
            K, dist,
            flags=cv2.SOLVEPNP_ITERATIVE
        )

        if not ok:
            return False

        R, _ = cv2.Rodrigues(rvec)
        tvec = tvec.reshape(3,1)

        self.extrinsics[cam_name] = {
            "rvec": rvec,
            "tvec": tvec,
            "R": R
        }

        P = K @ np.hstack((R, tvec))
        self.P_cache[cam_name] = P

        print(f"[Calib] extrinsics computed for {cam_name}: tvec={tvec.ravel()}")

        return True

    def cam_to_world(self, cam_name, X_cam):
        """
        X_cam: (3,) or (3,N) or (N,3)
        Returns world coords in (3,) or (N,3)
        Formula: X_cam = R * X_world + t  ->  X_world = R.T @ (X_cam - t)
        """
        e = self.extrinsics.get(cam_name)
        if e is None:
            raise RuntimeError("Calibrator: extrinsic not ready for " + cam_name)
        R = e['R']; t = e['tvec']
        X = np.asarray(X_cam, dtype=np.float64)
        if X.ndim == 1 and X.shape[0] == 3:
            Xc = X.reshape(3,1)
            Xw = R.T @ (Xc - t)
            return Xw[:,0]
        if X.ndim == 2 and X.shape[1] == 3:
            Xc = X.T  # 3xN
            Xw = R.T @ (Xc - t)
            return Xw.T
        if X.ndim == 2 and X.shape[0] == 3:
            Xc = X
            Xw = R.T @ (Xc - t)
            return Xw.T
        raise ValueError("Invalid X_cam shape: " + str(X.shape))

    def projection_matrix(self, cam_name):
        return self.P_cache.get(cam_name, None)

# --- Triangulation + reprojection checks ---
def undistort_points_to_normalized(K, dist, pts):
    """returns Nx2 normalized image points ready for triangulate (but keep as pixel K applied later)"""
    if len(pts) == 0:
        return np.zeros((0,2), dtype=np.float64)
    pts = np.array(pts, dtype=np.float64).reshape(-1,1,2)
    und = cv2.undistortPoints(pts, K, dist, P=K)  # returns Nx1x2 in pixel coords (reprojected)
    und = und.reshape(-1,2)
    return und

def triangulate_pair(P1, P2, pt1, pt2):
    """
    pt1,pt2: (x,y) pixel coordinates (not homogeneous). P1,P2: 3x4 projection matrices.
    Returns 3D point (x,y,z) in world coordinates (since P are K*[R|t] world->cam), after converting homog.
    """
    pts1 = np.array(pt1, dtype=np.float64).reshape(2,1)
    pts2 = np.array(pt2, dtype=np.float64).reshape(2,1)
    Xh = cv2.triangulatePoints(P1, P2, pts1, pts2)  # 4x1
    X = (Xh[:3,0] / Xh[3,0]).astype(np.float64)
    return X

def reprojection_error(P, X_world, observed_xy):
    """Project X_world using P and compute pixel error to observed_xy."""
    Xh = np.zeros((4,1), dtype=np.float64)
    Xh[:3,0] = X_world
    Xh[3,0] = 1.0
    proj = P @ Xh  # 3x1
    proj_xy = proj[:2,0] / proj[2,0]
    return float(np.linalg.norm(proj_xy - np.array(observed_xy, dtype=np.float64)))

def match_and_triangulate(camera_candidates, calibrator, max_pairs=128, reproj_thresh_px=6.0, depth_min=0.03, depth_max=10.0):
    cams = list(camera_candidates.keys())
    if len(cams) < 2:
        return []

    # choose primary pair (prefer fixed pair if both exist)
    pair = None
    if 'kreo1' in camera_candidates and 'kreo2' in camera_candidates:
        pair = ('kreo1', 'kreo2')
    else:
        pair = (cams[0], cams[1])

    c1, c2 = pair
    cand1 = sorted(camera_candidates[c1], key=lambda x: -x.get('area',1))[:8]
    cand2 = sorted(camera_candidates[c2], key=lambda x: -x.get('area',1))[:8]
    P1 = calibrator.projection_matrix(c1)
    P2 = calibrator.projection_matrix(c2)
    if P1 is None or P2 is None:
        return []

    results = []
    for a in cand1:
        for b in cand2:
            if len(results) >= max_pairs:
                break
            x1 = a['centroid']
            x2 = b['centroid']
            try:
                Xw = triangulate_pair(P1, P2, x1, x2)  # 3D world
            except Exception:
                continue
            # check depth in camera frames: compute X_cam = R*X_world + t  (use extrinsics of cam)
            # quickly get depth for cam1 and cam2
            e1 = calibrator.extrinsics[c1]
            R1, t1 = e1['R'], e1['tvec']
            Xcam1 = R1 @ Xw.reshape(3,1) + t1
            depth1 = float(Xcam1[2,0])
            e2 = calibrator.extrinsics[c2]
            R2, t2 = e2['R'], e2['tvec']
            Xcam2 = R2 @ Xw.reshape(3,1) + t2
            depth2 = float(Xcam2[2,0])
            if not (depth_min < depth1 < depth_max and depth_min < depth2 < depth_max):
                continue
            # reprojection errors:
            err1 = reprojection_error(P1, Xw, x1)
            err2 = reprojection_error(P2, Xw, x2)
            tot = err1 + err2
            if tot > (2*reproj_thresh_px):
                continue
            results.append({'pt': Xw, 'reproj_err': tot, 'pair': (c1,c2), 'cam_pixels': (x1,x2), 'depths':(depth1,depth2)})
    # sort by reprojection error
    results = sorted(results, key=lambda r: r['reproj_err'])
    return results

# ---------- ZMQ subscriber ----------
ctx = zmq.Context()
sub = ctx.socket(zmq.SUB)
sub.connect(ZMQ_ADDR)
sub.setsockopt(zmq.RCVHWM, 1)
sub.setsockopt(zmq.CONFLATE, 1)  # keep only last message
sub.setsockopt(zmq.LINGER, 0)

# ---- ACTIVE FLUSH ----
flushed = 0
while True:
    try:
        sub.recv_multipart(flags=zmq.NOBLOCK)
        flushed += 1
    except zmq.Again:
        break
if flushed > 0:
    print(f"[Subscriber] Flushed {flushed} stale messages.")
for t in SUB_TOPICS:
    sub.setsockopt(zmq.SUBSCRIBE, t)

# per-camera structures
frames = {}
fps_windows = defaultdict(lambda: deque())
frame_queues = {t.decode(): queue.Queue(maxsize=1) for t in SUB_TOPICS}
detect_cache = {}        # cam -> detection dict
detect_lock = threading.Lock()
tag_threads={}
ball_threads={}

# shared outputs
calibrator = StaticCalibrator(TAG_WORLD_MAP, TAG_SIZES)
shared_camera_candidates = defaultdict(list)  # cam_name -> list of ball detections per frame
shared_robot_poses = {}  # cam_name -> latest robot tag poses in world coords
last_triangulated_ball = None


for t in SUB_TOPICS:
    cam_name = t.decode()
    tag_threads[cam_name] = AprilTagThread(cam_name,frame_queues[cam_name],detect_cache,detect_lock,calibrator,shared_robot_poses)
    ball_threads[cam_name] = BallThread(cam_name,frame_queues[cam_name],detect_cache,detect_lock,shared_camera_candidates)

    tag_threads[cam_name].start()
    ball_threads[cam_name].start()

print("[Subscriber] connected, waiting for frames... (Press ESC to exit)")

last_show = time.time()
# ---------- Main loop ----------
try:
    while True:
        parts=recv_latest(sub)
        if parts is None: continue

        topic=parts[0]; cam=topic.decode()
        if len(parts)>=3:
            ts_part=parts[1]; jpg_part=parts[2]
        else:
            ts_part=None; jpg_part=parts[1]

        recv_t=time.time()
        try: cam_ts=float(ts_part.decode()) if ts_part else recv_t
        except: cam_ts=recv_t

        img=cv2.imdecode(np.frombuffer(jpg_part,np.uint8),cv2.IMREAD_COLOR)
        if img is None: continue

        fps=update_fps(cam,cam_ts)
        frames[cam]={"img":img,"cam_ts":cam_ts,"fps":fps}

        # push frame to both detector queues
        fq=frame_queues[cam]
        try: fq.get_nowait()
        except: pass
        try: fq.put_nowait((img.copy(),cam_ts))
        except: pass

        with detect_lock:
            ready = all(c in calibrator.extrinsics for c in [t.decode() for t in SUB_TOPICS])
        if ready:
            with detect_lock:
                tri_results = match_and_triangulate(shared_camera_candidates, calibrator)
            if len(tri_results) > 0:
                best = tri_results[0]
                Xw = best['pt']
                last_triangulated_ball = {'pos': Xw, 'err': best['reproj_err'], 'ts': time.time(), 'pair': best['pair'], 'depths':best['depths']}
            with detect_lock:
                current_time = time.time()
                b_x, b_y, b_z = "", "", "" 
                
                # Optional: Only log ball if seen recently (e.g., within last 0.5s)
                if last_triangulated_ball is not None:
                    # You can add a time check here if you want: if current_time - last_triangulated_ball['ts'] < 0.5:
                    pos = last_triangulated_ball['pos']
                    b_x, b_y, b_z = pos[0], pos[1], pos[2]

                # 2. GET TAG 4 POSITION
                t4_x, t4_y, t4_z = "", "", ""
                if 4 in shared_robot_poses:
                    pos4 = shared_robot_poses[4]['world_pos']
                    t4_x, t4_y, t4_z = pos4[0], pos4[1], pos4[2]

                # 3. GET TAG 5 POSITION
                t5_x, t5_y, t5_z = "", "", ""
                if 5 in shared_robot_poses:
                    pos5 = shared_robot_poses[5]['world_pos']
                    t5_x, t5_y, t5_z = pos5[0], pos5[1], pos5[2]

                # Write the row
                csv_writer.writerow([
                    current_time, 
                    b_x, b_y, b_z, 
                    t4_x, t4_y, t4_z, 
                    t5_x, t5_y, t5_z
                ])

        # visual output if both cams available
        if all(k in frames for k in [t.decode() for t in SUB_TOPICS]):
            cams=[t.decode() for t in SUB_TOPICS]
            L=frames[cams[0]]; R=frames[cams[1]]
            drift_ms=abs(L["cam_ts"]-R["cam_ts"])*1000.0

            def overlay(F,cam_name):
                im=F["img"].copy()
                y=20
                cv2.putText(im,f"{cam_name}",(10,y),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,32,20),2)
                cv2.putText(im,f"FPS:{F['fps']:.1f}",(10,y+26),cv2.FONT_HERSHEY_SIMPLEX,0.6,(14,117,5),2)
                cv2.putText(im,f"cam_ts:{fmt_ts(F['cam_ts'])}",(10,y+52),cv2.FONT_HERSHEY_SIMPLEX,0.5,(5,12,117),1)

                with detect_lock:
                    block=detect_cache.get(cam_name,{})

                    # draw AprilTags
                    if "tags" in block and block["tags"]["ids"] is not None:
                        cs=block["tags"]["corners"]
                        ids=block["tags"]["ids"]
                        cv2.aruco.drawDetectedMarkers(im,cs,ids)

                    # draw balls
                    if "balls" in block:
                        for i,d in enumerate(block["balls"]):
                            x,y,w,h=d["bbox"]
                            cx,cy=d["centroid"]
                            color=d["color"]
                            if color=="orange": bc=(0,200,255)
                            elif color=="purple": bc=(200,0,200)
                            else: bc=(0,200,200)
                            cv2.rectangle(im,(x,y),(x+w,y+h),bc,2)
                            cv2.circle(im,(cx,cy),4,(0,0,255),-1)
                            cv2.putText(im,f"{color}:{i}",(x,y-6),
                                        cv2.FONT_HERSHEY_SIMPLEX,0.5,bc,2)
                    
                    # draw robot world positions if available
                    text_y = 80
                    if shared_robot_poses:
                        for tid,info in shared_robot_poses.items():
                            pos = info.get('world_pos')
                            if pos is not None:
                                tx = f"Tag{tid} world: {pos[0]:.3f},{pos[1]:.3f},{pos[2]:.3f}"
                                cv2.putText(im, tx, (10, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1)
                                text_y += 18

                    # draw last triangulated ball if this is left camera (just show on left)
                    if cam_name == cams[0] and last_triangulated_ball is not None:
                        p = last_triangulated_ball['pos']
                        tx = f"Ball world: {p[0]:.3f},{p[1]:.3f},{p[2]:.3f} err:{last_triangulated_ball['err']:.2f}"
                        cv2.putText(im, tx, (10, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)

                return im

            if VISUALIZE and (time.time()-last_show)>(1.0/DISPLAY_FPS):
                last_show=time.time()
                left_im=overlay(L,cams[0])
                right_im=overlay(R,cams[1])

                h=max(left_im.shape[0],right_im.shape[0])
                right_res=cv2.resize(right_im,(left_im.shape[1],h))
                tile=np.hstack([left_im,right_res])

                cv2.putText(tile,f"Drift:{drift_ms:.1f}ms",(10,20),
                            cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2)
                
                cv2.putText(tile, f"Host now: {fmt_ts(time.time())}", (10, 44), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

                cv2.imshow("Combined Detection",tile)
            elif not VISUALIZE:
                with detect_lock:
                    tag_summary = []
                    parts_status = []
                    for c in cams:
                        block = detect_cache.get(c,{})

                        if "tags" in block and block["tags"]["ids"] is not None:
                            ids=block["tags"]["ids"]
                            tag_summary.append(f"{c}: {ids.flatten().tolist()}")
                        elif "tags" in block:
                            tag_summary.append(f"{c}: No tags")
                        else:
                            tag_summary.append(f"{c}: No detection data")
                        if "balls" in block:
                            counts = { "orange":0, "purple":0, "unknown":0 }
                            for i,d in enumerate(block["balls"]):
                                counts[d.get("color","unknown")] = counts.get(d.get("color","unknown"),0) + 1
                            parts_status.append(f"{c}: Orange: {counts['orange']} Purple: {counts['purple']}")
                        else:
                            parts_status.append(f"{c}:NoBall")    
                status = (
                    f"Drift {drift_ms:.1f} ms | "
                    f"{cams[0]} ts: {fmt_ts(L['cam_ts'])} | "
                    f"{cams[1]} ts: {fmt_ts(R['cam_ts'])} | "
                    f"Host now: {fmt_ts(time.time())} | "
                    f"{cams[0]} FPS: {L['fps']:.1f} | "
                    f"{cams[1]} FPS: {R['fps']:.1f} | "
                    f"Tags:" + ",".join(tag_summary)
                )
                sys.stdout.write("\r" + status + " " * 20 + " | ".join(parts_status) + " " * 20)
                sys.stdout.flush()

        if cv2.waitKey(1)&0xFF==27:
            break

except KeyboardInterrupt:
    pass

finally:
    for t in tag_threads.values(): t.stop()
    for t in ball_threads.values(): t.stop()

    if 'csv_file' in locals() and csv_file:
        csv_file.close()
        print("[Logging] CSV file closed.")
    time.sleep(0.1)
    cv2.destroyAllWindows()
    sub.close()
    ctx.term()
    print("Exit clean.")

[Logging] Started logging to data_log_1763916069.csv
[kreo1] AprilTagThread started
[kreo1] BallThread started
[kreo2] AprilTagThread started
[kreo2] BallThread started
[Subscriber] connected, waiting for frames... (Press ESC to exit)
[INFO] Loaded calibrated camera parameters
[Calib] extrinsics computed for kreo2: tvec=[-0.41997071  0.72816583  2.27687833]
[WARN] kreo2 tag4 mean_corner_z negative (-0.0420). Flipping Z to 0.0420
tid: 4, pos: [0.20305228 0.36071879 0.04197505]
[WARN] kreo2 tag4 mean_corner_z negative (-0.0246). Flipping Z to 0.0246
[INFO] kreo2 tag4 measured z 0.025, nominal 0.075
tid: 4, pos: [0.20251939 0.34804397 0.02459195]
[WARN] kreo2 tag4 mean_corner_z negative (-0.0370). Flipping Z to 0.0370
tid: 4, pos: [0.20285106 0.35708139 0.0369986 ]
[WARN] kreo2 tag5 mean_corner_z negative (-0.2034). Flipping Z to 0.2034
[INFO] kreo2 tag5 measured z 0.203, nominal 0.075
tid: 5, pos: [0.42820369 0.49286883 0.2033574 ]
[WARN] kreo2 tag4 mean_corner_z negative (-0.0565). Flip

In [10]:
import cv2, zmq, numpy as np, time, threading, queue, traceback, sys, os, csv, copy
from collections import deque, defaultdict

# ---------- Config ----------
ZMQ_ADDR = "tcp://localhost:5555"
SUB_TOPICS = [b"kreo1", b"kreo2"]
FPS_WINDOW = 1.0        
DISPLAY_FPS = 20
VISUALIZE = True     

STATIC_TAG_IDS = [0,1,2,3]
TAG_POSITIONS = {
    0: np.array([0.9, 0.0, 0.0], dtype=float),
    1: np.array([0.0, 0.0, 0.0], dtype=float),
    2: np.array([0.9, 0.9, 0.0], dtype=float),
    3: np.array([0.0, 0.9, 0.0], dtype=float)
}
TAG_SIZES = {0: 0.099, 1: 0.096, 2: 0.096, 3: 0.096, 4: 0.096, 5: 0.096}
CALIB_FRAMES = 30
CALIB_DIR = "../calibration/"

# Tuning for Matching
MAX_TIME_DIFF = 0.08  # 50ms max difference between camera frames to allow triangulation

# ---------- Async CSV Logging ----------
log_filename = f"../../data/triangulation_logs/data_log_{int(time.time())}.csv"
log_queue = queue.Queue()

def logger_worker():
    """Background thread to handle file writes without blocking the camera loop."""
    with open(log_filename, mode='w', newline='') as f:
        writer = csv.writer(f)
        header = [
            "timestamp", 
            "ball_x", "ball_y", "ball_z", 
            "tag4_x", "tag4_y", "tag4_z", 
            "tag5_x", "tag5_y", "tag5_z"
        ]
        writer.writerow(header)
        print(f"[Logging] Started logging to {log_filename}")
        
        while True:
            row = log_queue.get()
            if row is None: break
            writer.writerow(row)
            # f.flush() # Optional: Un-comment if you need real-time safety at cost of speed

log_thread = threading.Thread(target=logger_worker, daemon=True)
log_thread.start()

# ---------------- APRILTAG CONFIG ----------------
DICT_TYPE = cv2.aruco.DICT_APRILTAG_36h11
def create_april_detector():
    aruco_dict = cv2.aruco.getPredefinedDictionary(DICT_TYPE)
    params = cv2.aruco.DetectorParameters()
    params.adaptiveThreshWinSizeMin = 3
    params.adaptiveThreshWinSizeMax = 35
    params.adaptiveThreshWinSizeStep = 2
    params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX
    params.cornerRefinementWinSize = 7
    params.cornerRefinementMaxIterations = 50
    params.cornerRefinementMinAccuracy = 0.01
    params.minMarkerPerimeterRate = 0.02
    params.maxMarkerPerimeterRate = 6.0
    params.polygonalApproxAccuracyRate = 0.02
    params.adaptiveThreshConstant = 7
    return cv2.aruco.ArucoDetector(aruco_dict, params)

# --- USER TUNED HSV VALUES ---
HSV_CONFIG = {
    "kreo1": {
        "orange": {'hmin': 0, 'smin': 116, 'vmin': 160, 'hmax': 12, 'smax': 197, 'vmax': 255},
        # Keep purple as backup or ignore if not used
        "purple": {'hmin': 113,'smin': 78,  'vmin': 3,   'hmax': 129, 'smax': 255, 'vmax': 255}
    },
    "kreo2": {
        "orange": {'hmin': 0, 'smin': 79, 'vmin': 181, 'hmax': 12, 'smax': 255, 'vmax': 255},
        "purple": {'hmin': 113,'smin': 78,  'vmin': 3,   'hmax': 129, 'smax': 255, 'vmax': 255}
    }
}

MIN_AREA = 100
MAX_AREA = 8000
CIRCULARITY_MIN = 0.5
ASPECT_RATIO_MIN = 0.6
ASPECT_RATIO_MAX = 1.6
MAX_DETECTIONS_PER_CAM = 12

# ---------- Helpers ----------
def fmt_ts(ts):
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ts)) + f".{int((ts%1)*1000):03d}"

def recv_latest(sub):
    msg = None
    while True:
        try:
            msg = sub.recv_multipart(flags=zmq.NOBLOCK)
        except zmq.Again:
            break
    return msg

def update_fps(camera, cam_ts):
    dq = fps_windows[camera]
    dq.append(cam_ts)
    while dq and (cam_ts - dq[0]) > FPS_WINDOW:
        dq.popleft()
    fps = len(dq) / FPS_WINDOW
    return fps

def load_camera_calib(cam_name):
    path = os.path.join(CALIB_DIR, f'camera_calibration_{cam_name}.npz')
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    calib = np.load(path)
    camera_matrix = calib["cameraMatrix"]
    dist_coeffs = calib['distCoeffs']
    print("[INFO] Loaded calibrated camera parameters")
    return camera_matrix, dist_coeffs


def build_tag_world_map_from_centers(tag_centers, tag_sizes):
    out = {}
    for tid, center in tag_centers.items():
        size = tag_sizes.get(tid, tag_sizes.get(1))
        half = float(size) / 2.0
        local = np.array([
            [-half,  half, 0.0],
            [ half,  half, 0.0],
            [ half, -half, 0.0],
            [-half, -half, 0.0],
        ], dtype=np.float64)
        corners_world = (local + center.reshape(1,3)).astype(np.float64)
        out[tid] = corners_world
    return out

TAG_WORLD_MAP = build_tag_world_map_from_centers(TAG_POSITIONS, TAG_SIZES)

# ---------- Color masking helpers ----------
def hsv_mask_from_vals(bgr_img, hsvVals):
    hsv = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)
    lower = np.array([hsvVals['hmin'], hsvVals['smin'], hsvVals['vmin']], dtype=np.uint8)
    upper = np.array([hsvVals['hmax'], hsvVals['smax'], hsvVals['vmax']], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.medianBlur(mask, 5)
    return mask

def postprocess_mask(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    m = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kernel, iterations=2)
    return m

def find_candidate_contours(mask):
    if mask is None: return []
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidates = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < MIN_AREA or area > MAX_AREA: continue
        perim = cv2.arcLength(c, True)
        if perim <= 0: continue
        circularity = 4 * np.pi * area / (perim * perim)
        x,y,w,h = cv2.boundingRect(c)
        aspect = float(w)/float(h) if h>0 else 0.0
        if circularity < CIRCULARITY_MIN: continue
        if aspect < ASPECT_RATIO_MIN or aspect > ASPECT_RATIO_MAX: continue
        candidates.append((c, area, (int(x),int(y),int(w),int(h))))
    candidates.sort(key=lambda d: d[1], reverse=True)
    return candidates

def estimate_pose_apriltag(corners, tag_size, cam_mtx, cam_dist):
    half = tag_size / 2.0
    objp = np.array([
        [-half,  half, 0.0],
        [ half,  half, 0.0],
        [ half, -half, 0.0],
        [-half, -half, 0.0]
    ], dtype=np.float32)
    imgp = corners.reshape(4,2).astype(np.float32)
    ok, rvec, tvec = cv2.solvePnP(objp, imgp, cam_mtx, cam_dist, flags=cv2.SOLVEPNP_ITERATIVE)
    if not ok: raise RuntimeError("solvePnP failed")
    R, _ = cv2.Rodrigues(rvec)
    T = np.eye(4); T[:3, :3] = R; T[:3, 3] = tvec.reshape(3)
    return T

# APRILTAG THREAD
class AprilTagThread(threading.Thread):
    def __init__(self, cam_name, frame_queue, detect_cache, lock, calibrator, shared_robot_poses):
        super().__init__(daemon=True)
        self.cam_name = cam_name
        self.frame_queue = frame_queue
        self.detect_cache = detect_cache
        self.lock = lock
        self.detector = create_april_detector()
        self.stop_flag = False
        self.calibrator = calibrator
        self.shared_robot_poses = shared_robot_poses
    
    def run(self):
        print(f"[{self.cam_name}] AprilTagThread started")
        while not self.stop_flag:
            try: frame,ts = self.frame_queue.get(timeout=0.1)
            except queue.Empty: continue
            try:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                corners, ids, _ = self.detector.detectMarkers(gray)
                det={"corners":corners,"ids":ids,"ts":ts,"det_time":time.time()}

                with self.lock:
                    if self.cam_name not in self.detect_cache: self.detect_cache[self.cam_name]={}
                    self.detect_cache[self.cam_name]["tags"]=det

                if ids is not None:
                    self.calibrator.add_detection(self.cam_name,ids,corners,ts)
                    # Try to compute extrinsic immediately if we see a known tag
                    self.calibrator.try_compute_extrinsic(self.cam_name)

                    # (Existing robot tag logic maintained...)
                    if self.cam_name in self.calibrator.extrinsics:
                        K, dist = self.calibrator.load_intrinsics(self.cam_name)
                        for i,idarr in enumerate(ids):
                            tid = int(idarr[0])
                            if tid in (4,5):
                                corners_i = np.array(corners[i]).reshape(4,2)
                                T_tag_cam = estimate_pose_apriltag(corners_i, TAG_SIZES[tid], K, dist)
                                tag_origin_world = self.calibrator.cam_to_world(self.cam_name, T_tag_cam[:3,3].reshape(3,))
                                
                                with self.lock:
                                    self.shared_robot_poses[tid] = {
                                        'world_pos': tag_origin_world,
                                        'cam': self.cam_name,
                                        'ts': ts
                                    }
            except Exception:
                traceback.print_exc()
        print(f"[DETECT-{self.cam_name}] AprilTag Detector thread stopped")

    def stop(self): self.stop_flag = True

# BALL THREAD
class BallThread(threading.Thread):
    def __init__(self, cam_name, frame_queue, detect_cache, lock, shared_camera_candidates):
        super().__init__(daemon=True)
        self.cam_name = cam_name
        self.frame_queue = frame_queue
        self.detect_cache = detect_cache
        self.lock = lock
        self.stop_flag = False
        self.shared_camera_candidates = shared_camera_candidates
    
    def run(self):
        print(f"[{self.cam_name}] BallThread started")
        cam_hsv = HSV_CONFIG.get(self.cam_name, HSV_CONFIG.get("kreo1"))
        while not self.stop_flag:
            try: frame,ts=self.frame_queue.get(timeout=0.1)
            except queue.Empty: continue
            try:
                mo=hsv_mask_from_vals(frame,cam_hsv["orange"])
                mp=hsv_mask_from_vals(frame,cam_hsv["orange"])
                mc = cv2.bitwise_or(mo,mp)
                mc = postprocess_mask(mc)
                cand = find_candidate_contours(mc)

                dets = []
                for c,area,(x,y,w,h) in cand[:MAX_DETECTIONS_PER_CAM]:
                    M = cv2.moments(c)
                    if M["m00"] != 0: cx = int(M["m10"]/M["m00"]); cy = int(M["m01"]/M["m00"])
                    else: cx = x + w//2; cy = y + h//2
                    
                    s_or = int(np.count_nonzero(mo[y:y+h,x:x+w])) if mo is not None else 0
                    s_pu = int(np.count_nonzero(mp[y:y+h,x:x+w])) if mp is not None else 0
                    print(s_or,s_pu)

                    if s_or>=s_pu and s_or>0: col="orange"
                    elif s_pu>s_or and s_pu>0: col="purple"
                    else: col="unknown"

                    dets.append({
                        "bbox":(x,y,w,h),
                        "centroid":(cx,cy),
                        "area":float(area),
                        "color":col,
                        "ts":ts
                    })
                with self.lock:
                    if self.cam_name not in self.detect_cache: self.detect_cache[self.cam_name]={}
                    self.detect_cache[self.cam_name]["balls"] = dets
                    self.shared_camera_candidates[self.cam_name] = dets
            except Exception:
                traceback.print_exc()
        print(f"[{self.cam_name}] BallDetectorThread stopped")
    def stop(self): self.stop_flag = True

# --- IMPROVED CALIBRATOR ---
class StaticCalibrator:
    def __init__(self, tag_world_map, tag_size_map):
        self.tag_world_map = tag_world_map
        self.tag_size_map = tag_size_map
        self.obs = defaultdict(list)
        self.extrinsics = {}
        self.frame_count = defaultdict(int)
        self.K_cache = {}
        self.dist_cache = {}

    def load_intrinsics(self, cam_name):
        if cam_name in self.K_cache: return self.K_cache[cam_name], self.dist_cache[cam_name]
        camera_matrix, dist_coeffs = load_camera_calib(cam_name)
        self.K_cache[cam_name] = camera_matrix
        self.dist_cache[cam_name] = dist_coeffs
        return camera_matrix, dist_coeffs

    def add_detection(self, cam_name, ids, corners, ts):
        if ids is None: return
        self.frame_count[cam_name] += 1
        for i, idarr in enumerate(ids):
            tid = int(idarr[0])
            if tid in STATIC_TAG_IDS:
                c = np.array(corners[i]).reshape(4,2).astype(np.float64)
                self.obs[cam_name].append((tid, c, ts))

    def try_compute_extrinsic(self, cam_name):
        if cam_name in self.extrinsics: return True
        if self.frame_count.get(cam_name, 0) < CALIB_FRAMES: return False

        # FIX 1: Removed hardcoded tag IDs. Use ANY visible tag defined in world map.
        obs_list = list(reversed(self.obs.get(cam_name, [])))
        
        target_tag = None
        use_corners = None

        for (tid, corners, ts) in obs_list:
            if tid in self.tag_world_map:
                target_tag = tid
                use_corners = corners.reshape(4,2).astype(np.float64)
                break # Use the most recent valid tag
        
        if use_corners is None: return False # No known tags seen

        try: K, dist = self.load_intrinsics(cam_name)
        except: return False

        obj_corners = np.array(self.tag_world_map[target_tag], dtype=np.float64)
        ok, rvec, tvec = cv2.solvePnP(obj_corners, use_corners, K, dist, flags=cv2.SOLVEPNP_ITERATIVE)
        if not ok: return False

        R, _ = cv2.Rodrigues(rvec)
        tvec = tvec.reshape(3,1)
        self.extrinsics[cam_name] = {"rvec": rvec, "tvec": tvec, "R": R}
        print(f"[Calib] {cam_name} extrinsics computed using Tag {target_tag}")
        return True

    def cam_to_world(self, cam_name, X_cam):
        e = self.extrinsics.get(cam_name)
        if e is None: raise RuntimeError("Calibrator: extrinsic not ready for " + cam_name)
        R = e['R']; t = e['tvec']
        X = np.asarray(X_cam, dtype=np.float64)
        if X.ndim == 1: Xc = X.reshape(3,1); Xw = R.T @ (Xc - t); return Xw[:,0]
        else: Xc = X.T; Xw = R.T @ (Xc - t); return Xw.T

    def get_norm_projection_matrix(self, cam_name):
        # Returns [R|t] (3x4)
        e = self.extrinsics.get(cam_name)
        if e is None: return None
        return np.hstack((e['R'], e['tvec']))
    
def match_and_triangulate(camera_candidates, calibrator, reproj_thresh_px=15.0):
    cams = list(camera_candidates.keys())
    if len(cams) < 2: return []
    
    pair = ('kreo1', 'kreo2') if 'kreo1' in cams and 'kreo2' in cams else (cams[0], cams[1])
    c1, c2 = pair
    
    # Filter candidates (sort by area)
    cand1 = sorted(camera_candidates[c1], key=lambda x: -x.get('area',1))[:8]
    cand2 = sorted(camera_candidates[c2], key=lambda x: -x.get('area',1))[:8]
    #print(cand1,cand2)

    # Get Calibration Data
    if c1 not in calibrator.extrinsics or c2 not in calibrator.extrinsics: return []
    
    K1, D1 = calibrator.load_intrinsics(c1)
    K2, D2 = calibrator.load_intrinsics(c2)
    P1_norm = calibrator.get_norm_projection_matrix(c1) # [R1|t1]
    P2_norm = calibrator.get_norm_projection_matrix(c2) # [R2|t2]

    results = []
    
    for a in cand1:
        for b in cand2:
            # FIX 3: Check Color and Timestamp
            if a['color'] != b['color']: continue
            
            dt = abs(a['ts'] - b['ts'])
            if dt > MAX_TIME_DIFF: continue

            # FIX 2: Undistort Points before Triangulation
            pt1_in = np.array(a['centroid'], dtype=float).reshape(-1,1,2)
            pt2_in = np.array(b['centroid'], dtype=float).reshape(-1,1,2)
            
            # Undistort to Normalized Coordinates (x, y) where z=1
            pt1_norm = cv2.undistortPoints(pt1_in, K1, D1, P=None)
            pt2_norm = cv2.undistortPoints(pt2_in, K2, D2, P=None)

            pt1_norm = pt1_norm.reshape(-1, 2).T  # Becomes (2, 1)
            pt2_norm = pt2_norm.reshape(-1, 2).T  # Becomes (2, 1)

            # Triangulate in World Frame using Normalized Projection Matrices
            Xh = cv2.triangulatePoints(P1_norm, P2_norm, pt1_norm, pt2_norm)
            w = Xh[3]
            if abs(w) < 1e-6: continue # Skip invalid points at infinity
            Xw = (Xh[:3] / w).flatten()

            # 2. Project (using K and Dist)
            img_pt1, _ = cv2.projectPoints(Xw.reshape(1,3), calibrator.extrinsics[c1]['rvec'], calibrator.extrinsics[c1]['tvec'], K1, D1)
            img_pt2, _ = cv2.projectPoints(Xw.reshape(1,3), calibrator.extrinsics[c2]['rvec'], calibrator.extrinsics[c2]['tvec'], K2, D2)

            err1 = np.linalg.norm(img_pt1.flatten() - np.array(a['centroid']))
            err2 = np.linalg.norm(img_pt2.flatten() - np.array(b['centroid']))
            tot_err = err1 + err2

            if tot_err < reproj_thresh_px:
                results.append({
                    'pt': Xw, 
                    'reproj_err': tot_err, 
                    'pair': (c1,c2), 
                    'depths': (float(Xw[2].item()), float(Xw[2].item())),
                    'ts': max(a['ts'], b['ts']) # Use latest ts
                })

    results.sort(key=lambda r: r['reproj_err'])
    return results


# ---------- ZMQ subscriber ----------
ctx = zmq.Context()
sub = ctx.socket(zmq.SUB)
sub.connect(ZMQ_ADDR)
sub.setsockopt(zmq.RCVHWM, 4)
sub.setsockopt(zmq.CONFLATE, 1)
sub.setsockopt(zmq.LINGER, 0)

# Active flush
flushed = 0
while True:
    try: sub.recv_multipart(flags=zmq.NOBLOCK); flushed += 1
    except zmq.Again: break
if flushed > 0: print(f"[Subscriber] Flushed {flushed} stale messages.")
for t in SUB_TOPICS: sub.setsockopt(zmq.SUBSCRIBE, t)

frames = {}
fps_windows = defaultdict(lambda: deque())
frame_queues = {t.decode(): queue.Queue(maxsize=1) for t in SUB_TOPICS}
detect_cache = {}
detect_lock = threading.Lock()
tag_threads={}
ball_threads={}

calibrator = StaticCalibrator(TAG_WORLD_MAP, TAG_SIZES)
shared_camera_candidates = defaultdict(list)
shared_robot_poses = {}
last_triangulated_ball = None
last_ekf_ts = -1.0

for t in SUB_TOPICS:
    cam_name = t.decode()
    tag_threads[cam_name] = AprilTagThread(cam_name,frame_queues[cam_name],detect_cache,detect_lock,calibrator,shared_robot_poses)
    ball_threads[cam_name] = BallThread(cam_name,frame_queues[cam_name],detect_cache,detect_lock,shared_camera_candidates)
    tag_threads[cam_name].start()
    ball_threads[cam_name].start()

print("[Subscriber] connected.")

last_show = time.time()

try:
    while True:
        parts=recv_latest(sub)
        if parts is None: continue

        topic=parts[0]; cam=topic.decode()
        jpg_part=parts[2] if len(parts)>=3 else parts[1]
        ts_part=parts[1] if len(parts)>=3 else None

        recv_t=time.time()
        try: cam_ts=float(ts_part.decode()) if ts_part else recv_t
        except: cam_ts=recv_t

        img=cv2.imdecode(np.frombuffer(jpg_part,np.uint8),cv2.IMREAD_COLOR)
        if img is None: continue

        fps=update_fps(cam,cam_ts)
        frames[cam]={"img":img,"cam_ts":cam_ts,"fps":fps}
        
        fq=frame_queues[cam]
        try: fq.get_nowait()
        except: pass
        try: fq.put_nowait((img.copy(),cam_ts))
        except: pass

        # Check calibration ready
        candidates_copy = None
        calib_ready = False
        with detect_lock:
            if all(c in calibrator.extrinsics for c in [t.decode() for t in SUB_TOPICS]):
                calib_ready = True
                candidates_copy = copy.deepcopy(shared_camera_candidates)

        # 2. PROCESS DATA (Slow, lock released)
        if calib_ready and candidates_copy:
            # Pass the COPY, not the shared dict
            tri_results = match_and_triangulate(candidates_copy, calibrator)
            last_triangulated_ball = None
            if len(tri_results) > 0:
                best = tri_results[0]
                Xw = best['pt']
                last_triangulated_ball = {'pos': Xw, 'err': best['reproj_err'], 'ts': best['ts'], 'pair': best['pair'], 'depths':best['depths']}

            # Logging (Now Async)
            current_time = time.time()
            b_x, b_y, b_z = "", "", ""
            if last_triangulated_ball and (current_time - last_triangulated_ball['ts'] < 0.5):
                pos = last_triangulated_ball['pos']
                b_x, b_y, b_z = pos[0], pos[1], pos[2]
            
            t4_x, t4_y, t4_z = "", "", ""
            if 4 in shared_robot_poses:
                pos4 = shared_robot_poses[4]['world_pos']
                t4_x, t4_y, t4_z = pos4[0], pos4[1], pos4[2]
            
            t5_x, t5_y, t5_z = "", "", ""
            if 5 in shared_robot_poses:
                pos5 = shared_robot_poses[5]['world_pos']
                t5_x, t5_y, t5_z = pos5[0], pos5[1], pos5[2]

            log_queue.put([current_time, b_x, b_y, b_z, t4_x, t4_y, t4_z, t5_x, t5_y, t5_z])

        # Visualization (Same as before)
        if all(k in frames for k in [t.decode() for t in SUB_TOPICS]):
            cams=[t.decode() for t in SUB_TOPICS]
            L=frames[cams[0]]; R=frames[cams[1]]
            drift_ms=abs(L["cam_ts"]-R["cam_ts"])*1000.0
            
            if VISUALIZE and (time.time()-last_show)>(1.0/DISPLAY_FPS):
                last_show=time.time()

                def overlay(F,cam_name):
                    im=F["img"].copy()
                    y=20
                    cv2.putText(im,f"{cam_name}",(10,y),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,32,20),2)
                    cv2.putText(im,f"FPS:{F['fps']:.1f}",(10,y+26),cv2.FONT_HERSHEY_SIMPLEX,0.6,(14,117,5),2)
                    cv2.putText(im,f"cam_ts:{fmt_ts(F['cam_ts'])}",(10,y+52),cv2.FONT_HERSHEY_SIMPLEX,0.5,(5,12,117),1)

                    with detect_lock:
                        block=detect_cache.get(cam_name,{})

                        # draw AprilTags
                        if "tags" in block and block["tags"]["ids"] is not None:
                            cs=block["tags"]["corners"]
                            ids=block["tags"]["ids"]
                            cv2.aruco.drawDetectedMarkers(im,cs,ids)

                        # draw balls
                        if "balls" in block:
                            for i,d in enumerate(block["balls"]):
                                x,y,w,h=d["bbox"]
                                cx,cy=d["centroid"]
                                color=d["color"]
                                if color=="orange": bc=(0,200,255)
                                elif color=="purple": bc=(200,0,200)
                                else: bc=(0,200,200)
                                cv2.rectangle(im,(x,y),(x+w,y+h),bc,2)
                                cv2.circle(im,(cx,cy),4,(0,0,255),-1)
                                cv2.putText(im,f"{color}:{i}",(x,y-6),
                                            cv2.FONT_HERSHEY_SIMPLEX,0.5,bc,2)
                        
                        # draw robot world positions if available
                        text_y = 80
                        if shared_robot_poses:
                            for tid,info in shared_robot_poses.items():
                                pos = info.get('world_pos')
                                if pos is not None:
                                    tx = f"Tag{tid} world: {pos[0]:.3f},{pos[1]:.3f},{pos[2]:.3f}"
                                    cv2.putText(im, tx, (10, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1)
                                    text_y += 18

                        # draw last triangulated ball if this is left camera (just show on left)
                        if last_triangulated_ball is not None:
                            p = last_triangulated_ball['pos']
                            tx = f"Ball world: {p[0]:.3f},{p[1]:.3f},{p[2]:.3f} err:{last_triangulated_ball['err']:.2f}"
                            cv2.putText(im, tx, (10, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)

                    return im
                
                left_im=overlay(L,cams[0])
                right_im=overlay(R,cams[1])

                h=max(left_im.shape[0],right_im.shape[0])
                right_res=cv2.resize(right_im,(left_im.shape[1],h))
                tile=np.hstack([left_im,right_res])

                cv2.putText(tile,f"Drift:{drift_ms:.1f}ms",(10,20),
                            cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),2)
                
                cv2.putText(tile, f"Host now: {fmt_ts(time.time())}", (10, 44), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

                cv2.imshow("Combined Detection",tile)
            elif not VISUALIZE:
                with detect_lock:
                    tag_summary = []
                    parts_status = []
                    for c in cams:
                        block = detect_cache.get(c,{})

                        if "tags" in block and block["tags"]["ids"] is not None:
                            ids=block["tags"]["ids"]
                            tag_summary.append(f"{c}: {ids.flatten().tolist()}")
                        elif "tags" in block:
                            tag_summary.append(f"{c}: No tags")
                        else:
                            tag_summary.append(f"{c}: No detection data")
                        if "balls" in block:
                            counts = { "orange":0, "purple":0, "unknown":0 }
                            for i,d in enumerate(block["balls"]):
                                counts[d.get("color","unknown")] = counts.get(d.get("color","unknown"),0) + 1
                            parts_status.append(f"{c}: Orange: {counts['orange']} Purple: {counts['purple']}")
                        else:
                            parts_status.append(f"{c}:NoBall")    
                status = (
                    f"Drift {drift_ms:.1f} ms | "
                    f"{cams[0]} ts: {fmt_ts(L['cam_ts'])} | "
                    f"{cams[1]} ts: {fmt_ts(R['cam_ts'])} | "
                    f"Host now: {fmt_ts(time.time())} | "
                    f"{cams[0]} FPS: {L['fps']:.1f} | "
                    f"{cams[1]} FPS: {R['fps']:.1f} | "
                    f"Tags:" + ",".join(tag_summary)
                )
                sys.stdout.write("\r" + status + " " * 20 + " | ".join(parts_status) + " " * 20)
                sys.stdout.flush()
        if cv2.waitKey(1)&0xFF==27: break

except KeyboardInterrupt: pass
finally:
    log_queue.put(None)
    log_thread.join()
    for t in tag_threads.values(): t.stop()
    for t in ball_threads.values(): t.stop()
    time.sleep(0.1)
    cv2.destroyAllWindows()
    sub.close()
    ctx.term()
    print("Exit clean.")


[Logging] Started logging to ../../data/triangulation_logs/data_log_1763918611.csv
[kreo1] AprilTagThread started
[kreo1] BallThread started
[kreo2] AprilTagThread started
[kreo2] BallThread started
[Subscriber] connected.
[INFO] Loaded calibrated camera parameters
[Calib] kreo2 extrinsics computed using Tag 0
[INFO] Loaded calibrated camera parameters
[Calib] kreo1 extrinsics computed using Tag 1
[kreo1] BallDetectorThread stopped
[kreo2] BallDetectorThread stopped
[DETECT-kreo1] AprilTag Detector thread stopped
[DETECT-kreo2] AprilTag Detector thread stopped
Exit clean.


In [2]:
import cv2, zmq, numpy as np, time, threading, queue, traceback, sys, os, csv, copy
from collections import deque, defaultdict

# ---------- Config ----------
ZMQ_ADDR = "tcp://localhost:5555"
SUB_TOPICS = [b"kreo1", b"kreo2"]
FPS_WINDOW = 1.0        
DISPLAY_FPS = 30        
VISUALIZE = True        

# SCALING CONFIG
BALL_DETECTION_SCALE = 0.5  # Downscale for speed (Ball)
TAG_DETECTION_SCALE = 1.0   # Full res for accuracy (Tags)

LOG_DIR = "../../data/triangulation_logs"
LOG_FILENAME = f"{LOG_DIR}/log_{int(time.time())}.csv"
CALIB_DIR = "../calibration/"

# HSV Config
HSV_CONFIG = {
    "kreo1": { "orange": {'hmin': 0, 'smin': 116, 'vmin': 160, 'hmax': 12, 'smax': 197, 'vmax': 255} },
    "kreo2": { "orange": {'hmin': 0, 'smin': 110, 'vmin': 181, 'hmax': 10, 'smax': 255, 'vmax': 255} }
}
DEFAULT_HSV = {'hmin': 0, 'smin': 100, 'vmin': 100, 'hmax': 25, 'smax': 255, 'vmax': 255}

# Detector parameters
BASE_MIN_AREA = 100    
BASE_MAX_AREA = 20000  
CIRCULARITY_MIN = 0.5
ASPECT_RATIO_MIN = 0.6 
ASPECT_RATIO_MAX = 1.6   
MAX_DETECTIONS_PER_CAM = 5 

# 3D / TRIANGULATION CONFIG
STATIC_TAG_IDS = [0,1,2,3]
TAG_POSITIONS = {
    0: np.array([0.9, 0.0, 0.0], dtype=float),
    1: np.array([0.0, 0.0, 0.0], dtype=float),
    2: np.array([0.9, 0.9, 0.0], dtype=float),
    3: np.array([0.0, 0.9, 0.0], dtype=float)
}
TAG_SIZES = {0: 0.099, 1: 0.096, 2: 0.096, 3: 0.096, 4: 0.096, 5: 0.096}
CALIB_FRAMES = 30
MAX_TIME_DIFF = 0.05  # Max ms diff for triangulation pairing

# AprilTag Config
DICT_TYPE = cv2.aruco.DICT_APRILTAG_36h11
def create_april_detector():
    aruco_dict = cv2.aruco.getPredefinedDictionary(DICT_TYPE)
    params = cv2.aruco.DetectorParameters()
    params.adaptiveThreshWinSizeMin = 3
    params.adaptiveThreshWinSizeMax = 35
    params.adaptiveThreshWinSizeStep = 2
    params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX
    params.cornerRefinementWinSize = 7
    params.cornerRefinementMaxIterations = 50
    params.cornerRefinementMinAccuracy = 0.01
    params.minMarkerPerimeterRate = 0.02
    params.maxMarkerPerimeterRate = 6.0
    params.polygonalApproxAccuracyRate = 0.02
    params.adaptiveThreshConstant = 7
    return cv2.aruco.ArucoDetector(aruco_dict, params)

class StaticCalibrator:
    def __init__(self, tag_world_map, tag_size_map):
        self.tag_world_map = tag_world_map
        self.tag_size_map = tag_size_map
        self.obs = defaultdict(list)
        self.extrinsics = {}
        self.frame_count = defaultdict(int)
        self.P_cache = {}              # cam_name -> projection matrix K@[R|t]
        self.K_cache = {}
        self.dist_cache = {}
        self.calib_observations = defaultdict(list)  # cam_name -> list of (rvec, tvec, reprojection_error)
        self.MAX_CALIB_OBS = 50  # Number of observations to collect
        self.REPROJ_ERROR_THRESHOLD = 2.0  

    def load_intrinsics(self, cam_name):
        if cam_name in self.K_cache: return self.K_cache[cam_name], self.dist_cache[cam_name]
        camera_matrix, dist_coeffs = load_camera_calib(cam_name)
        self.K_cache[cam_name] = camera_matrix
        self.dist_cache[cam_name] = dist_coeffs
        return camera_matrix, dist_coeffs

    def add_detection(self, cam_name, ids, corners, ts):
        if ids is None: return
        self.frame_count[cam_name] += 1
        for i, idarr in enumerate(ids):
            tid = int(idarr[0])
            if tid in STATIC_TAG_IDS:
                c = np.array(corners[i]).reshape(4,2).astype(np.float64)
                self.obs[cam_name].append((tid, c, ts))
    
    def compute_reprojection_error(self, obj_corners, img_corners, rvec, tvec, K, dist):
        """Calculate mean reprojection error in pixels"""
        projected, _ = cv2.projectPoints(obj_corners, rvec, tvec, K, dist)
        projected = projected.reshape(-1, 2)
        error = np.linalg.norm(projected - img_corners, axis=1).mean()
        return error

    def try_compute_extrinsic(self, cam_name):
        if cam_name in self.extrinsics: return True
        if self.frame_count.get(cam_name, 0) < CALIB_FRAMES: return False

        target_tag = None
        if cam_name == "kreo1":
            target_tag = 2
        elif cam_name == "kreo2":
            target_tag = 1
        else:
            return False

        try: 
            K, dist = self.load_intrinsics(cam_name)
        except: 
            return False

        # FIX 1: Collect multiple observations instead of just one
        obs_list = [(tid, corners, ts) for (tid, corners, ts) in self.obs.get(cam_name, []) 
                    if int(tid) == int(target_tag)]
        
        if len(obs_list) < 10:  # Need at least 10 good observations
            return False

        # FIX 2: Use all recent observations to compute multiple pose estimates
        obj_corners = np.array(self.tag_world_map[target_tag], dtype=np.float64)
        
        for (tid, corners, ts) in obs_list[-30:]:  # Use last 30 observations
            use_corners = corners.reshape(4, 2).astype(np.float64)
            
            # FIX 3: Use SOLVEPNP_ITERATIVE to match detect_apriltags.py
            ok, rvec, tvec = cv2.solvePnP(
                obj_corners, use_corners, K, dist, 
                flags=cv2.SOLVEPNP_ITERATIVE  # Changed from IPPE_SQUARE
            )
            
            if not ok: 
                continue
            
            # FIX 4: Calculate and filter by reprojection error
            reproj_error = self.compute_reprojection_error(
                obj_corners, use_corners, rvec, tvec, K, dist
            )
            
            if reproj_error < self.REPROJ_ERROR_THRESHOLD:
                self.calib_observations[cam_name].append({
                    'rvec': rvec.copy(),
                    'tvec': tvec.copy(),
                    'error': reproj_error,
                    'ts': ts
                })

        # FIX 5: Need enough good observations
        if len(self.calib_observations[cam_name]) < 10:
            return False

        # FIX 6: Robust averaging using median of rotation vectors and translation vectors
        # (Simple averaging works for small rotations; for production, use proper SO(3) averaging)
        valid_obs = sorted(self.calib_observations[cam_name], key=lambda x: x['error'])[:20]
        
        rvecs = np.array([obs['rvec'].flatten() for obs in valid_obs])
        tvecs = np.array([obs['tvec'].flatten() for obs in valid_obs])
        
        # Use median to be robust against outliers
        rvec_avg = np.median(rvecs, axis=0).reshape(3, 1)
        tvec_avg = np.median(tvecs, axis=0).reshape(3, 1)
        
        # Convert to rotation matrix
        R, _ = cv2.Rodrigues(rvec_avg)
        
        # Store averaged extrinsics
        self.extrinsics[cam_name] = {
            "rvec": rvec_avg, 
            "tvec": tvec_avg, 
            "R": R
        }
        
        P = np.hstack((R, tvec_avg))
        self.P_cache[cam_name] = P
        T = np.eye(4); T[:3, :3] = R.T; T[:3, 3] = -R.T @ tvec.reshape(3)

        print(f"[Calib] {cam_name} extrinsics locked using Tag {target_tag} {T}")
        return True

    def cam_to_world(self, cam_name, X_cam):
        e = self.extrinsics.get(cam_name)
        if e is None: raise RuntimeError("Calibrator: extrinsic not ready for " + cam_name)
        R = e['R']; t = e['tvec']
        X = np.asarray(X_cam, dtype=np.float64)
        if X.ndim == 1 and X.shape[0] == 3:
            Xc = X.reshape(3,1)
            Xw = R.T @ (Xc - t)
            return Xw[:,0]
        if X.ndim == 2 and X.shape[1] == 3:
            Xc = X.T  # 3xN
            Xw = R.T @ (Xc - t)
            return Xw.T
        if X.ndim == 2 and X.shape[0] == 3:
            Xc = X
            Xw = R.T @ (Xc - t)
            return Xw.T
        raise ValueError("Invalid X_cam shape: " + str(X.shape))

    def get_norm_projection_matrix(self, cam_name):
        return self.P_cache.get(cam_name, None)


# ---------- Logging Setup ----------
if not os.path.exists(LOG_DIR): os.makedirs(LOG_DIR)
log_queue = queue.Queue()

def logger_worker():
    try:
        with open(LOG_FILENAME, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                "timestamp","camera", 
                "ball_2d_x", "ball_2d_y", "ball_2d_area",
                "ball_3d_x", "ball_3d_y", "ball_3d_z",
                "tag4_x", "tag4_y", "tag4_z",
                "tag5_x", "tag5_y", "tag5_z",
                "detected_kreo1", "detected_kreo2"
            ])
            while True:
                entry = log_queue.get()
                if entry is None: break
                writer.writerow(entry)
                log_queue.task_done()
    except Exception as e: print(f"[LOGGER ERROR] {e}")

log_thread = threading.Thread(target=logger_worker, daemon=True)
log_thread.start()



# ---------- Helpers ----------
def fmt_ts(ts):
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(ts)) + f".{int((ts%1)*1000):03d}"

def recv_latest(sub):
    msg = None
    while True:
        try:
            msg = sub.recv_multipart(flags=zmq.NOBLOCK)
        except zmq.Again:
            break
    return msg

def update_fps(camera, cam_ts):
    dq = fps_windows[camera]
    dq.append(cam_ts)
    # pop older than window
    while dq and (cam_ts - dq[0]) > FPS_WINDOW:
        dq.popleft()
    fps = len(dq) / FPS_WINDOW
    return fps

def load_camera_calib(cam_name):
    path = os.path.join(CALIB_DIR, f'camera_calibration_{cam_name}.npz')
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    calib = np.load(path)
    camera_matrix = calib["cameraMatrix"]
    dist_coeffs = calib['distCoeffs']
    print("[INFO] Loaded calibrated camera parameters")
    return camera_matrix, dist_coeffs

def build_tag_world_map_from_centers(tag_centers, tag_sizes):
    out = {}
    for tid, center in tag_centers.items():
        size = tag_sizes.get(tid, tag_sizes.get(1))
        half = float(size) / 2.0
        local = np.array([
            [-half,  half, 0.0],
            [ half,  half, 0.0],
            [ half, -half, 0.0],
            [-half, -half, 0.0],
        ], dtype=np.float64)
        corners_world = (local + center.reshape(1,3)).astype(np.float64)
        out[tid] = corners_world
    return out

TAG_WORLD_MAP = build_tag_world_map_from_centers(TAG_POSITIONS, TAG_SIZES)

def estimate_pose_apriltag(corners, tag_size, cam_mtx, cam_dist):
    half = tag_size / 2.0
    objp = np.array([
        [-half,  half, 0.0],
        [ half,  half, 0.0],
        [ half, -half, 0.0],
        [-half, -half, 0.0]
    ], dtype=np.float32)
    imgp = corners.reshape(4,2).astype(np.float32)
    ok, rvec, tvec = cv2.solvePnP(objp, imgp, cam_mtx, cam_dist, flags=cv2.SOLVEPNP_IPPE_SQUARE)
    if not ok: raise RuntimeError("solvePnP failed")
    R, _ = cv2.Rodrigues(rvec)
    T = np.eye(4); T[:3, :3] = R; T[:3, 3] = tvec.reshape(3)
    return T

def get_orange_mask(bgr_img, hsv_dict):
    hsv = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)
    lower = np.array([hsv_dict['hmin'], hsv_dict['smin'], hsv_dict['vmin']], dtype=np.uint8)
    upper = np.array([hsv_dict['hmax'], hsv_dict['smax'], hsv_dict['vmax']], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower, upper)
    return cv2.GaussianBlur(mask, (5, 5), 0)

def find_ball_contours(mask, min_area, max_area):
    if mask is None: return []
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidates = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < min_area or area > max_area: continue
        x,y,w,h = cv2.boundingRect(c)
        aspect = float(w)/float(h) if h > 0 else 0
        if ASPECT_RATIO_MIN>aspect or aspect > ASPECT_RATIO_MAX: continue
        perim = cv2.arcLength(c, True)
        if perim == 0: continue
        circularity = 4 * np.pi * area / (perim * perim)
        if circularity >= CIRCULARITY_MIN:
            candidates.append({"bbox": (x,y,w,h), "area": area, "centroid": (x+w//2, y+h//2)})
    candidates.sort(key=lambda d: d["area"], reverse=True)
    return candidates

# ---------- Shared Data (Thread Safe) ----------
# Stores the latest tag detection result for each camera
shared_tag_data = {
    "kreo1": {"tag4": {"detected": False, "x":"", "y":""}, "tag5": {"detected": False, "x":"", "y":""}, "viz": []},
    "kreo2": {"tag4": {"detected": False, "x":"", "y":""}, "tag5": {"detected": False, "x":"", "y":""}, "viz": []}
}
# 3D Data (Populated by Threads and Main Loop)
shared_3d_poses = {
    "ball": None, # (x, y, z)
    4: None,      # (x, y, z)
    5: None       # (x, y, z)
}

# Candidates for Ball Triangulation (Written by BallThread, Read by Main)
shared_ball_candidates = defaultdict(list)
shared_ball_status = {"kreo1": False, "kreo2": False}
shared_ball_history = {
    "kreo1": deque(maxlen=30),   # ~ 30 recent candidates
    "kreo2": deque(maxlen=30)
}
shared_data_lock = threading.Lock()
viz_cache = {}
viz_lock = threading.Lock()
# Global Calibrator Instance
calibrator = StaticCalibrator(TAG_WORLD_MAP, TAG_SIZES)

# ---------- THREAD 1: TAG DETECTOR (Slower, High Accuracy) ----------
class TagDetectorThread(threading.Thread):
    def __init__(self, cam_name, frame_queue):
        super().__init__(daemon=True)
        self.cam_name = cam_name
        self.frame_queue = frame_queue
        self.aruco_detector = create_april_detector()
        self.stop_flag = False

    def run(self):
        print(f"[{self.cam_name}] Tag Thread started.")
        while not self.stop_flag:
            try:
                # Get raw bytes (Tag thread decodes independently to avoid blocking Ball thread)
                jpg_bytes, cam_ts = self.frame_queue.get(timeout=0.1)
                
                # Decode
                frame = cv2.imdecode(np.frombuffer(jpg_bytes, np.uint8), cv2.IMREAD_COLOR)
                if frame is None: continue

                # Detect (Full Res)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                corners, ids, _ = self.aruco_detector.detectMarkers(gray)
                
                viz_tags = []
                local_tag4_det = False
                local_tag5_det = False

                if ids is not None:
                    calibrator.add_detection(self.cam_name, ids, corners, cam_ts)
                    calibrator.try_compute_extrinsic(self.cam_name)

                    ids_flat = ids.flatten()
                    for i, tag_id in enumerate(ids_flat):
                        viz_tags.append({"id": tag_id, "corners": corners[i]})
                        
                        # 2. Compute 3D for Moving Tags (4 & 5)
                        # Only if extrinsic is locked
                        if tag_id in [4, 5] and self.cam_name in calibrator.extrinsics:
                            try:
                                K, dist = calibrator.load_intrinsics(self.cam_name)
                                c_corners = np.array(corners[i]).reshape(4,2)
                                T_tag_cam = estimate_pose_apriltag(c_corners, TAG_SIZES[tag_id], K, dist)
                                tag_world = calibrator.cam_to_world(self.cam_name, T_tag_cam[:3,3].reshape(3,))
                                
                                with shared_data_lock:
                                    shared_3d_poses[tag_id] = tag_world
                                    if tag_id == 4: local_tag4_det = True
                                    if tag_id == 5: local_tag5_det = True
                            except Exception as e:
                                pass # Pose estimation failed for this frame
                
                # Update 2D Shared
                with shared_data_lock:
                    shared_tag_data[self.cam_name] = {
                        "tag4": {"detected": local_tag4_det},
                        "tag5": {"detected": local_tag5_det},
                        "viz": viz_tags
                    }
            except queue.Empty:
                pass
            except Exception as e:
                print(f"[TAG-ERR-{self.cam_name}]", e)

    def stop(self):
        self.stop_flag = True

# ---------- THREAD 2: BALL DETECTOR (Fast, Low Latency) ----------
class BallDetectorThread(threading.Thread):
    def __init__(self, cam_name, frame_queue):
        super().__init__(daemon=True)
        self.cam_name = cam_name
        self.frame_queue = frame_queue
        self.min_area_scaled = BASE_MIN_AREA * (BALL_DETECTION_SCALE**2)
        self.max_area_scaled = BASE_MAX_AREA * (BALL_DETECTION_SCALE**2)
        self.hsv_vals = HSV_CONFIG.get(cam_name, {}).get("orange", DEFAULT_HSV)
        self.stop_flag = False
        self.fps_dq = deque()

    def run(self):
        print(f"[{self.cam_name}] Ball Thread started.")
        while not self.stop_flag:
            try:
                jpg_bytes, cam_ts = self.frame_queue.get(timeout=0.1)
                
                # Decode
                frame = cv2.imdecode(np.frombuffer(jpg_bytes, np.uint8), cv2.IMREAD_COLOR)
                if frame is None: continue

                # Resize for Speed
                if BALL_DETECTION_SCALE != 1.0:
                    frame_small = cv2.resize(frame, None, fx=BALL_DETECTION_SCALE, fy=BALL_DETECTION_SCALE, interpolation=cv2.INTER_NEAREST)
                else:
                    frame_small = frame

                # Detect Ball
                mask = get_orange_mask(frame_small, self.hsv_vals)
                balls = find_ball_contours(mask, self.min_area_scaled, self.max_area_scaled)
                
                ball_data = {"detected": False, "x": "", "y": "", "area": ""}
                viz_ball = None
                candidates_for_triangulation = []

                if balls:
                    b = balls[0] 
                    scale_inv = 1.0 / BALL_DETECTION_SCALE
                    bx, by, bw, bh = b["bbox"]
                    real_x = int(bx * scale_inv); real_y = int(by * scale_inv)
                    real_w = int(bw * scale_inv); real_h = int(bh * scale_inv)
                    real_area = int(b["area"] * (scale_inv**2))
                    cx, cy = real_x + real_w//2, real_y + real_h//2
                    
                    ball_data = {"detected": True, "x": cx, "y": cy, "area": real_area}
                    viz_ball = {"bbox": (real_x, real_y, real_w, real_h), "centroid": (cx, cy)}
                    
                    # Prepare data for match_and_triangulate
                    candidates_for_triangulation.append({
                        "centroid": (cx, cy),
                        "area": real_area,
                        "color": "orange",
                        "ts": cam_ts
                    })
                # Update Shared Memory
                with shared_data_lock:
                    # Update candidates for this camera
                    shared_ball_candidates[self.cam_name] = candidates_for_triangulation
                    shared_ball_status[self.cam_name] = ball_data["detected"]
                    if candidates_for_triangulation:
                        # store a lightweight record: (ts, centroid, area, color)
                        for c in candidates_for_triangulation:
                            shared_ball_history[self.cam_name].append({
                                "ts": c["ts"],
                                "centroid": c["centroid"],
                                "area": c.get("area", 0),
                                "color": c.get("color", "")
                            })
                    
                    # Read GLOBAL 3D Data (Calculated by Main Loop or Tag Thread)
                    # Note: There might be a 1-frame delay for Ball 3D, which is acceptable for decoupled logging
                    ball_3d = shared_3d_poses.get("ball")
                    tag4_3d = shared_3d_poses.get(4)
                    tag5_3d = shared_3d_poses.get(5)
                    tags_viz = shared_tag_data[self.cam_name]["viz"]
                    
                    # Read detection status of OTHER camera for logging
                    det_k1 = shared_ball_status["kreo1"]
                    det_k2 = shared_ball_status["kreo2"]
                
                # --- LOGGING (Driven by this thread) ---
                b3d = ball_3d if ball_3d is not None else ["", "", ""]
                t4d = tag4_3d if tag4_3d is not None else ["", "", ""]
                t5d = tag5_3d if tag5_3d is not None else ["", "", ""]

                log_queue.put([
                    f"{cam_ts:.3f}", self.cam_name,
                    ball_data["x"], ball_data["y"], ball_data["area"],
                    b3d[0], b3d[1], b3d[2],
                    t4d[0], t4d[1], t4d[2],
                    t5d[0], t5d[1], t5d[2],
                    det_k1, det_k2
                ])

                # FPS Calc
                self.fps_dq.append(time.time())
                while self.fps_dq and (self.fps_dq[-1] - self.fps_dq[0]) > FPS_WINDOW:
                    self.fps_dq.popleft()
                fps = len(self.fps_dq) / FPS_WINDOW

                if VISUALIZE:
                    with viz_lock:
                        viz_cache[self.cam_name] = {
                            "img": frame,
                            "ball": viz_ball,
                            "tags": tags_viz,
                            "fps": fps
                        }

            except queue.Empty:
                pass
            except Exception as e:
                print(f"[BALL-ERR-{self.cam_name}]", e)
    
    def stop(self):
        self.stop_flag = True

def match_and_triangulate(camera_candidates, calibrator, reproj_thresh_px=15.0):
    cams = list(camera_candidates.keys())
    if len(cams) < 2: return []
    
    pair = ('kreo1', 'kreo2') if 'kreo1' in cams and 'kreo2' in cams else (cams[0], cams[1])
    c1, c2 = pair
    
    # # Filter candidates (sort by area)
    # cand1 = sorted(camera_candidates[c1], key=lambda x: -x.get('area',1))[:8]
    # cand2 = sorted(camera_candidates[c2], key=lambda x: -x.get('area',1))[:8]

    # Get Calibration Data
    if c1 not in calibrator.extrinsics or c2 not in calibrator.extrinsics: return []
    
    K1, D1 = calibrator.load_intrinsics(c1)
    K2, D2 = calibrator.load_intrinsics(c2)
    P1_norm = calibrator.get_norm_projection_matrix(c1) # [R1|t1]
    P2_norm = calibrator.get_norm_projection_matrix(c2) # [R2|t2]

    results = []

    with shared_data_lock:
        hist1 = list(shared_ball_history[c1])
        hist2 = list(shared_ball_history[c2])
    
    for a in hist1:
        # for each candidate in hist1 find candidates in hist2 within MAX_TIME_DIFF
        closest = None
        best_dt = 1e9
        for b in hist2:
            if a['color'] != b['color']: 
                continue
            dt = abs(a['ts'] - b['ts'])
            if dt <= MAX_TIME_DIFF and dt < best_dt:
                best_dt = dt
                closest = b
        if closest is None:
            continue

        pt1_in = np.array(a['centroid'], dtype=float).reshape(-1,1,2)
        pt2_in = np.array(closest['centroid'], dtype=float).reshape(-1,1,2)
            
            # Undistort to Normalized Coordinates (x, y) where z=1
        pt1_norm = cv2.undistortPoints(pt1_in, K1, D1, P=None)
        pt2_norm = cv2.undistortPoints(pt2_in, K2, D2, P=None)

        pt1_norm = pt1_norm.reshape(-1, 2).T  
        pt2_norm = pt2_norm.reshape(-1, 2).T 

        # Triangulate in World Frame
        Xh = cv2.triangulatePoints(P1_norm, P2_norm, pt1_norm, pt2_norm)
        w = Xh[3]
        if abs(w) < 1e-6: continue
        Xw = (Xh[:3] / w).flatten()

        # Reproject to verify
        img_pt1, _ = cv2.projectPoints(Xw.reshape(1,3), calibrator.extrinsics[c1]['rvec'], calibrator.extrinsics[c1]['tvec'], K1, D1)
        img_pt2, _ = cv2.projectPoints(Xw.reshape(1,3), calibrator.extrinsics[c2]['rvec'], calibrator.extrinsics[c2]['tvec'], K2, D2)

        err1 = np.linalg.norm(img_pt1.flatten() - np.array(a['centroid']))
        err2 = np.linalg.norm(img_pt2.flatten() - np.array(b['centroid']))
        tot_err = err1 + err2

        if tot_err < reproj_thresh_px:
            results.append({
                'pt': Xw, 
                'reproj_err': tot_err, 
                'ts': max(a['ts'], b['ts']) 
            })
    results.sort(key=lambda r: r['reproj_err'])
    return results

# ---------- Main Execution ----------
ctx = zmq.Context()
sub = ctx.socket(zmq.SUB)
sub.connect(ZMQ_ADDR)
sub.setsockopt(zmq.RCVHWM, 4)
sub.setsockopt(zmq.CONFLATE, 1) 
sub.setsockopt(zmq.LINGER, 0)
for t in SUB_TOPICS: sub.setsockopt(zmq.SUBSCRIBE, t)

# Init Queues (2 per camera: 1 for Ball, 1 for Tag)
queues = {
    "kreo1": {"ball": queue.Queue(maxsize=1), "tag": queue.Queue(maxsize=1)},
    "kreo2": {"ball": queue.Queue(maxsize=1), "tag": queue.Queue(maxsize=1)}
}

threads = []
# Spin up threads
for cam in ["kreo1", "kreo2"]:
    # Ball Thread
    bt = BallDetectorThread(cam, queues[cam]["ball"])
    bt.start()
    threads.append(bt)
    
    # Tag Thread
    tt = TagDetectorThread(cam, queues[cam]["tag"])
    tt.start()
    threads.append(tt)

print(f"[System] Decoupled Detection Started. Logging to {LOG_FILENAME}")

last_show = time.time()

try:
    while True:
        # High Speed Ingestion Loop
        try:
            parts = recv_latest(sub)
            if parts is None: continue
            topic = parts[0]
            cam = topic.decode()
            ts_part = parts[1] if len(parts) >= 3 else None
            jpg_part = parts[2] if len(parts) >= 3 else parts[1]
            try: cam_ts = float(ts_part.decode()) if ts_part else time.time()
            except: cam_ts = time.time()
            
            qs = queues.get(cam)
            if qs:
                try: qs["ball"].put_nowait((jpg_part, cam_ts))
                except queue.Full: pass
                try: qs["tag"].put_nowait((jpg_part, cam_ts))
                except queue.Full: pass
        except zmq.Again:
            time.sleep(0.0001)
        with shared_data_lock:
            ready = all(c in calibrator.extrinsics for c in [t.decode() for t in SUB_TOPICS])
        if not ready: continue
        # ----------------------------------------------------
        #  3D BALL CALCULATION & LOGGING
        # ----------------------------------------------------
        # We periodically check shared candidates to triangulate
        with shared_data_lock:
            current_ball_candidates = dict(shared_ball_candidates)

        # Perform Triangulation
        tri_results = match_and_triangulate(current_ball_candidates, calibrator)
        
        with shared_data_lock:
            if len(tri_results) > 0:
                res = tri_results[0]
                shared_3d_poses["ball"] = res['pt']
            else:
                # Optional: Decay old data or keep last known position? 
                # Keeping last known for now to avoid flickering, or set to None
                shared_3d_poses["ball"] = None

        # ----------------------------------------------------
        #  VISUALIZATION
        # ----------------------------------------------------
        curr_time = time.time()
        if VISUALIZE and (curr_time - last_show) > (1.0/DISPLAY_FPS):
            last_show = curr_time
            with viz_lock:
                has_data = all(c in viz_cache for c in ["kreo1", "kreo2"])
                if has_data:
                    def draw(cam_key):
                        d = viz_cache[cam_key]
                        im = d["img"].copy()
                        
                        if d["ball"]:
                            bx,by,bw,bh = d["ball"]["bbox"]
                            cx,cy = d["ball"]["centroid"]
                            cv2.rectangle(im, (bx,by), (bx+bw, by+bh), (0,165,255), 2)
                            cv2.circle(im, (cx,cy), 5, (0,0,255), -1)
                        
                        for tag in d["tags"]:
                             cv2.aruco.drawDetectedMarkers(im, [tag["corners"].astype(int)], np.array([[tag["id"]]]))
                        
                        # 3D Text Overlay
                        y_off = 60
                        if shared_3d_poses["ball"] is not None:
                            bp = shared_3d_poses["ball"]
                            cv2.putText(im, f"Ball 3D: {bp[0]:.2f}, {bp[1]:.2f}, {bp[2]:.2f}", (10, y_off), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
                            y_off += 25
                        
                        for tid in [4, 5]:
                            if shared_3d_poses[tid] is not None:
                                tp = shared_3d_poses[tid]
                                cv2.putText(im, f"Tag {tid} 3D: {tp[0]:.2f}, {tp[1]:.2f}, {tp[2]:.2f}", (10, y_off), 
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
                                y_off += 25
                        
                        if cam_key not in calibrator.extrinsics:
                             cv2.putText(im, f"CALIBRATING... {calibrator.frame_count.get(cam_key,0)}/{CALIB_FRAMES}", 
                                         (10, y_off), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

                        cv2.putText(im, f"FPS: {d['fps']:.1f}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                        return im

                    l_im = draw("kreo1")
                    r_im = draw("kreo2")
                    
                    h = min(l_im.shape[0], r_im.shape[0])
                    if l_im.shape[0] != h: l_im = cv2.resize(l_im, (int(l_im.shape[1]*h/l_im.shape[0]), h))
                    if r_im.shape[0] != h: r_im = cv2.resize(r_im, (int(r_im.shape[1]*h/r_im.shape[0]), h))
                    
                    cv2.imshow("Decoupled 3D", np.hstack([l_im, r_im]))

            if cv2.waitKey(1) & 0xFF == 27:
                break

except KeyboardInterrupt:
    pass
finally:
    for t in threads: t.stop()
    log_queue.put(None)
    log_thread.join()
    cv2.destroyAllWindows()
    sub.close()
    ctx.term()
    print("\nClean exit.")

[kreo1] Ball Thread started.
[kreo1] Tag Thread started.
[kreo2] Ball Thread started.
[kreo2] Tag Thread started.
[System] Decoupled Detection Started. Logging to ../../data/triangulation_logs/log_1764376313.csv
[INFO] Loaded calibrated camera parameters
[Calib] kreo1 extrinsics locked using Tag 2 [[-0.96485068  0.12657417 -0.23030881  1.17141528]
 [ 0.24477044  0.75182899 -0.61224227  1.73489094]
 [ 0.09565879 -0.64709516 -0.75638433  2.14219773]
 [ 0.          0.          0.          1.        ]]
[INFO] Loaded calibrated camera parameters
[Calib] kreo2 extrinsics locked using Tag 1 [[ 9.98395734e-01 -3.78634188e-02  4.20989301e-02  2.13801889e-01]
 [-5.66194627e-02 -6.73417907e-01  7.37090604e-01 -1.29911139e+00]
 [ 4.41403196e-04 -7.38291733e-01 -6.74481372e-01  2.07323077e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Clean exit.
